In [21]:
import argparse
from datasets import Dataset, load_dataset, concatenate_datasets,Features,Sequence,Value
import os
from huggingface_hub import login
import sys
from argparse import Namespace
import re
import openai
import json

In [27]:
features = Features({
    "repo": Value("string"),
    "pull_number": Value("int64"),
    "test_patch": Value("string"),
    "issue_numbers": Sequence(Value("string")),
    "instance_id": Value("string"),
    "problem_statement": Value("string"),
    "version": Value("string"),
    "base_commit": Value("string"),
    "patch": Value("string"),
    "created_at": Value("string"),
    "hints_text": Value("string"),
    "environment_setup_commit": Value("string"),
    "FAIL_TO_PASS": Sequence(Value("string")),
    "PASS_TO_PASS": Sequence(Value("string")),
    "FAIL_TO_FAIL": Sequence(Value("string")),  # 显式定义为字符串数组
    "PASS_TO_FAIL": Sequence(Value("string")),  # 显式定义为字符串数组
})
args = Namespace(
    dataset_name="r1v3r/RustGPT_Bench_verified",
    split="train"
)

token = os.getenv("HUGGING_FACE_HUB_TOKEN")
if not token:
    print("Error: Hugging Face access token not provided. Use --token or set the HUGGINGFACE_TOKEN environment variable.")
    sys.exit(1)

login(token=token)

# Load the dataset
print(f"Loading dataset '{args.dataset_name}' split '{args.split}' from Hugging Face...")
try:
    dataset = load_dataset(args.dataset_name, split=args.split)
except Exception as e:
    print(f"Error loading dataset: {e}")
    sys.exit(1)

dataset =  dataset.remove_columns('r_number')
print("After removing r_number:", dataset.column_names)
dataset.cast(features)
dataset.push_to_hub(args.dataset_name, split=args.split,token=token)


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Loading dataset 'r1v3r/RustGPT_Bench_verified' split 'train' from Hugging Face...
After removing r_number: ['repo', 'pull_number', 'instance_id', 'issue_numbers', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'environment_setup_commit', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'FAIL_TO_FAIL', 'PASS_TO_FAIL']


Casting the dataset:   0%|          | 0/95 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/r1v3r/RustGPT_Bench_verified/commit/b54f053c0ccb52f32cc5ffd13bb064fcc00ea18e', commit_message='Upload dataset', commit_description='', oid='b54f053c0ccb52f32cc5ffd13bb064fcc00ea18e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/r1v3r/RustGPT_Bench_verified', endpoint='https://huggingface.co', repo_type='dataset', repo_id='r1v3r/RustGPT_Bench_verified'), pr_revision=None, pr_num=None)

In [28]:
dataset = dataset.cast(features)

hyper_dataset = load_dataset("r1v3r/hyper_validated", split="train")
hyper_dataset = hyper_dataset.cast(features)
# 筛选数据
hyper_dataset = hyper_dataset.filter(
    lambda x: x["instance_id"] == "hyperium__hyper-3261" or x["instance_id"] == "hyperium__hyper-3275"
)

# print(hyper_dataset)
serde_dataset = load_dataset("r1v3r/serde_validated", split="train",features=features)
serde_dataset = serde_dataset.cast(features)
proc_macro2_dataset = load_dataset("r1v3r/proc-macro2_validated", split="train",features=features)
proc_macro2_dataset = proc_macro2_dataset.cast(features)

Casting the dataset:   0%|          | 0/6 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/2 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

In [29]:



temp = concatenate_datasets([hyper_dataset, serde_dataset, proc_macro2_dataset ,dataset])

temp.push_to_hub("r1v3r/RustGPT_Bench_100", token=token)



Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/r1v3r/RustGPT_Bench_100/commit/78b4725f2f7d1dedcb10cab0d7f0a5ac1e94f89b', commit_message='Upload dataset', commit_description='', oid='78b4725f2f7d1dedcb10cab0d7f0a5ac1e94f89b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/r1v3r/RustGPT_Bench_100', endpoint='https://huggingface.co', repo_type='dataset', repo_id='r1v3r/RustGPT_Bench_100'), pr_revision=None, pr_num=None)

修改某个值

In [ ]:
index = None
for i, instance in enumerate(dataset):
    if instance.get("instance_id") == "rayon-rs__rayon-986":
        index = i
        break

if index is not None:
    # Update the 'pull_number' for the found instance
    dataset = dataset.map(lambda example, idx: 
                          {**example,"pull_number": 986} if idx == index else example, 
                          with_indices=True)
    print(f"Updated 'pull_number' to 2 for instance with 'instance_id' 1.")
else:
    print("Error: No instance found with 'instance_id' == 1.")


统计buggy files

In [ ]:
def count_files_in_patch(example):
    patch_text = example['patch']
    # Use regex to find all lines that indicate file changes
    file_changes = re.findall(r'(?m)^(--- |\+\+\+ )([^\s]+)', patch_text)
    # Extract unique file paths
    unique_files = set(file for _, file in file_changes)
    # Return a dictionary with instance_id and the count of unique files
    return {'instance_id': example['instance_id'], 'file_number': len(unique_files)}

# Map the function over the dataset
results = dataset.map(count_files_in_patch, remove_columns=dataset.column_names)

# Convert the results to a dictionary for easy lookup
instance_file_map = {result['instance_id']: result['file_number'] for result in results}

# Print the resulting dictionary
for instance_id, file_number in instance_file_map.items():
    print(f"{instance_id}: {file_number}")


问gpt哪些是关于feature的

In [ ]:
import os
import openai
import json  # 用于保存 JSON 文件
from tqdm import tqdm
import pandas as pd

# 配置 OpenAI API
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.base_url = "https://api5.xhub.chat/v1/"

# 模型配置
model_name_or_path = "gpt-4o-mini"  # 或者 "gpt-3.5-turbo"
system_messages = "You are an AI assistant that categorizes tasks into 'Bug Fix' or 'Feature Development'."
temperature = 0.5
top_p = 0.9

def classify_problem_statement(problem_statement):
    """
    使用 OpenAI API 对 problem_statement 进行分类。

    Args:
        problem_statement (str): 问题陈述。

    Returns:
        str: 分类结果，可能的值为 'Bug Fix', 'Feature Development', 'Unknown', 'Error'。
    """
    user_message = (
        f"Categorize the following problem statement as either 'Bug Fix' or 'Feature Development':\n\n\"{problem_statement}\""
    )

    try:
        response = openai.chat.completions.create(
            model=model_name_or_path,
            messages=[
                {"role": "system", "content": system_messages},
                {"role": "user", "content": user_message},
            ],
            temperature=temperature,
            top_p=top_p,
        )

        # 根据 OpenAI API 的响应格式提取分类结果
        classification = response.choices[0].message.content.strip()
        print(f"Classification: {classification}")

        # 确保分类结果为预期的值
        if 'Bug Fix' in classification:
            return 'Bug Fix'
        elif 'Feature Development' in classification:
            return 'Feature Development'
        else:
            return 'Unknown'  # 无法确定的情况

    except Exception as e:
        print(f"Error processing problem_statement: {problem_statement}\nError: {e}")
        return 'Error'

"""
主函数，用于加载数据集，分类 problem_statement,并保存结果。
"""
# 加载数据集
# 初始化一个列表，用于存储分类为 'Feature Development' 的条目
feature_development_entries = []

# 迭代数据集中的每一项
print("Classifying problem statements...")
for example in tqdm(dataset, desc="Classifying"):
    problem_statement = example.get('problem_statement', None)
    instance_id = example.get('instance_id', None)  # 确保 'instance_id' 字段存在

    if pd.isna(problem_statement):
        classification = 'Unknown'
    else:
        classification = classify_problem_statement(problem_statement)

    # 如果分类为 'Feature Development'，将相关信息添加到列表中
    if classification == 'Feature Development':
        entry = {
            "instance_id": instance_id,
            "problem_statement": problem_statement,
            "response": classification
        }
        print("instance_id:", instance_id)
        feature_development_entries.append(entry)

# 保存分类为 'Feature Development' 的条目到 JSON 文件
output_json_path = "feature_development_entries.json"  # 指定输出 JSON 文件的路径
print(f"Saving Feature Development entries to {output_json_path}...")
with open(output_json_path, 'w', encoding='utf-8') as json_file:
    json.dump(feature_development_entries, json_file, ensure_ascii=False, indent=4)

print("分类完成，结果已保存。")




合并数据集

In [ ]:
repo_id = args.dataset_name
print(f"Uploading the cleaned dataset to Hugging Face repository '{repo_id}'...")

dataset.push_to_hub(repo_id, token=token)